In [7]:
import os
import cv2

def create_training_data(folder_path, classes, img_size):
    X_train = []
    y_train = []
    for category in classes:
        path = os.path.join(folder_path, category)
        class_num = classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (img_size, img_size))
                X_train.append(new_array)
                y_train.append(class_num)
            except Exception as e:
                pass
    print("Converted image shape: " + str(X_train[0].shape))
    return(X_train, y_train)

In [8]:
import numpy as np

train_data_path = "data"
classes = ["anger", "disgust", "fear", "happiness", "neutral", "sadness", "surprise"]


X_train, y_train = create_training_data(train_data_path, classes, 48)

X_train = np.array(X_train)
y_train = np.array(y_train)

Converted image shape: (48, 48, 3)


In [9]:
X_test = None
y_test = None
epochs = 2
batch_size = 256

number_of_labels = len(classes)
# Aplatir (vectoriser) les images
n_samples = len(X_train)
X_train = X_train.reshape((n_samples, -1))
input_shape = X_train.shape
print(input_shape)

(25409, 6912)


REDUCTION DE DIMENSIONS

In [10]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 150, whiten=True, random_state=42)
pca_transformee = pca.fit_transform(X_train)

# Enregistrer les donnees transformees dans un fichier csv
import pandas as pd

df = pd.DataFrame(pca_transformee)  # Créez un DataFrame avec les données transformées
df.to_csv('donnees_transformees.csv', index=False)  # Enregistrez le DataFrame dans un fichier CSV



CREATION DU MODELE SVM LINEAIRE

In [14]:
from sklearn import svm
from sklearn.calibration import CalibratedClassifierCV
from sklearn.pipeline import make_pipeline
model = svm.LinearSVC(max_iter=100000)
# Création d'un classificateur calibré pour pouvoir tracer la courbe ROC_AUC
calibrated_svc = CalibratedClassifierCV(model, method='sigmoid')
pipeline= make_pipeline(pca_transformee,calibrated_svc)


ENTRAINEMENT

In [19]:
from time import time
import datetime
import pandas as pd
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df,y_train,test_size=0.2,random_state=42)
print("Training ...")
start_time = time()
pipeline.fit(X_train, y_train)

end_time = time()
duration = end_time-start_time

print("Training duration: ", str(datetime.timedelta(seconds=duration)))

ValueError: Found input variables with inconsistent numbers of samples: [25409, 20327]

In [ ]:
support_vectors = model.support_vectors_
dec = model.decision_function([[1]])

In [ ]:
# from sklearn.metrics import classification_report
#
# print("TEST RESULTS")
# y_test_model = model.predict(X_test)
# print(classification_report(y_test, y_test_model))

y_pred = model.predict(X_test)
# Obtenir les probabilités de prédiction pour les données de test
y_pred_proba = calibrated_svc.predict_proba(X_test)

In [1]:
# from joblib import dump
#
# dump(model, 'model_svm.joblib')

# later, the model can be retrieved with "clf = load('filename.joblib')"